In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.ryerson.ca/calendar/2021-2022/programs/feas/mechanical/"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

# Scrape the first and second year courses

## 1. Collect course link texts for webdriver to click on

In [5]:
driver.get(url)

In [6]:
page_soup = soup(driver.page_source, 'lxml')

In [7]:
first_two_years_container = page_soup.findAll("div", {"class": "panel panel-default"})[2]
first_two_years_container

<div class="panel panel-default">
<!-- Accordion Heading -->
<div class="panel-heading container-clipboard">
<div class="panel-title">
<a aria-controls="accordion-content-1595938877357-mechanical-engineering---first-two-years-common" aria-expanded="false" data-toggle="collapse" href="#accordion-content-1595938877357-mechanical-engineering---first-two-years-common" id="accordion-1595938877357-mechanical-engineering---first-two-years-common" role="tab">
                            Mechanical Engineering - First Two Years Common
                        </a>
</div>
</div>
<!-- Accordion Body -->
<div aria-labelledby="accordion-1595938877357-mechanical-engineering---first-two-years-common" class="panel-collapse collapse" id="accordion-content-1595938877357-mechanical-engineering---first-two-years-common" role="tabpanel" tabindex="0">
<div class="panel-body">
<div class="1595938877357 stackparsys">
<div class="resTitle section">
<h2>
	
    1st &amp; 2nd Semester
    
	
</h2>
</div>
<div clas

In [8]:
link_texts = first_two_years_container.findAll("a", {"class": "qTipCourse"})
len(link_texts)

22

In [9]:
link_texts = [link_text.text for link_text in link_texts]
link_texts

['CEN 100',
 'CEN 199',
 'CHY 102',
 'MTH 140',
 'MTH 141',
 'PCS 211',
 'CPS 188',
 'ECN 801',
 'MEC 222',
 'MTH 240',
 'MTL 200',
 'PCS 125',
 'CMN 432',
 'MEC 323',
 'MEC 325',
 'MTH 425',
 'PCS 213',
 'EES 512',
 'MEC 309',
 'MEC 311',
 'MEC 322',
 'MTH 410']

## 2. Test run - try to scrape the first course

In [10]:
#open all the tabs first in order to click the links
open_tabs = driver.find_element_by_class_name("resAccordionOpenAll")
open_tabs

<selenium.webdriver.remote.webelement.WebElement (session="e3a3b8f7ee7bb08e4ec190d32de398f3", element="47e23f3a-795d-49b7-8b6f-229eff26f2b2")>

In [11]:
open_tabs.click()
time.sleep(2)
driver.find_element_by_link_text(link_texts[0]).click()
driver.current_url

'https://www.ryerson.ca/calendar/2021-2022/courses/common-engineering/CEN/100/'

In [12]:
page_soup = soup(driver.page_source, 'lxml')
container = page_soup.find("div", {"class": "resCalendarCourseEmbed resCalendarCourse"})
container

<div class="resCalendarCourseEmbed resCalendarCourse">
<h1>CEN 100</h1>
<h2>Introduction to Engineering</h2>
<div id="e0928d2e-8af9-49ec-9ac6-431a1009440f">
<div class="courseBlock">
<div class="courseDescription">This course is aimed at familiarizing the first year students with the basic information of the academic structure and expectations. Exposure to public and worker safety and the impact of engineering activities on health as well as safety standards and safety codes will be covered. The course also stresses integration with other first year courses. The principal objectives of the course are to provide a general introduction to the field of engineering; to convey the social, professional, and ethical responsibilities of engineers and why they are important to an engineering education; to introduce the undergraduate engineering programs available at Ryerson University; and to provide a general description of the skills needed to become a practicing engineer. Case studies in eng

In [13]:
course_code = container.find("h1").text.strip()
course_code

'CEN 100'

In [14]:
course_name = container.find("h2").text.strip()
course_name

'Introduction to Engineering'

In [15]:
course_desc = container.find("div", {"class": "courseDescription"}).text.strip()
course_desc

'This course is aimed at familiarizing the first year students with the basic information of the academic structure and expectations. Exposure to public and worker safety and the impact of engineering activities on health as well as safety standards and safety codes will be covered. The course also stresses integration with other first year courses. The principal objectives of the course are to provide a general introduction to the field of engineering; to convey the social, professional, and ethical responsibilities of engineers and why they are important to an engineering education; to introduce the undergraduate engineering programs available at Ryerson University; and to provide a general description of the skills needed to become a practicing engineer. Case studies in engineering are used to illustrate engineering fields and scientific principles.'

## 3. Test run successful. Implement automation script to scrape all courses

In [16]:
#scrape the first year courses
from selenium.common.exceptions import NoSuchElementException

course_codes = []
course_names = []
course_descs = []
counter = 0

driver.get("https://www.ryerson.ca/calendar/2021-2022/programs/feas/mechanical/")
driver.find_element_by_class_name("resAccordionOpenAll").click()
time.sleep(2)

for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()    
    time.sleep(2)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("div", {"class": "resCalendarCourseEmbed resCalendarCourse"})
    course_code = container.find("h1").text.strip()
    course_name = container.find("h2").text.strip()
    course_desc = container.find("div", {"class": "courseDescription"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(2)
    driver.find_element_by_class_name("resAccordionOpenAll").click()
    time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  CEN 100
Scraped  CEN 199
Scraped  CHY 102
Scraped  MTH 140
Scraped  MTH 141
Scraped  PCS 211
Scraped  CPS 188
Scraped  ECN 801
Scraped  MEC 222
Scraped  MTH 240
Scraped  MTL 200
Scraped  PCS 125
Scraped  CMN 432
Scraped  MEC 323
Scraped  MEC 325
Scraped  MTH 425
Scraped  PCS 213
Scraped  EES 512
Scraped  MEC 309
Scraped  MEC 311
Scraped  MEC 322
Scraped  MTH 410
Finished scraping 22 courses


# Scrape third and fourth year courses

## 1. Collect course link texts for webdriver to click on

In [17]:
#now update link_texts to hold the links of the years 3-4 courses
driver.get("https://www.ryerson.ca/calendar/2021-2022/programs/feas/mechanical/")
time.sleep(3)
page_soup = soup(driver.page_source, 'lxml')
upper_years_container = page_soup.findAll("div", {"class": "panel panel-default"})[3]
upper_years_container

<div class="panel panel-default">
<!-- Accordion Heading -->
<div class="panel-heading container-clipboard">
<div class="panel-title">
<a aria-controls="accordion-content-1595938877375-full-time--four-year-program" aria-expanded="false" data-toggle="collapse" href="#accordion-content-1595938877375-full-time--four-year-program" id="accordion-1595938877375-full-time--four-year-program" role="tab">
                            Full-Time, Four-Year Program
                        </a>
</div>
</div>
<!-- Accordion Body -->
<div aria-labelledby="accordion-1595938877375-full-time--four-year-program" class="panel-collapse collapse" id="accordion-content-1595938877375-full-time--four-year-program" role="tabpanel" tabindex="0">
<div class="panel-body">
<div class="1595938877375 stackparsys">
<div class="resText parbase section">
<div class="res-text richtextContent background-hidden" id="7aed7438-18b2-4735-bd5a-db9463622dae">
<p><img alt=" " class="hr-separator hr-separator-default" role="present

In [18]:
#need to filter out a liberal studies courses container
sub_containers = upper_years_container.findAll("div", {"class": "resTable parbase section"})
len(sub_containers)

9

In [19]:
sub_containers[3]

<div class="resTable parbase section">
<div class="scrollIndicator">
<i aria-hidden="true" class="fa fa-long-arrow-left"></i> Swipe to see more <i aria-hidden="true" class="fa fa-long-arrow-right"></i>
</div>
<div id="23d466d3-9a0b-4dcf-9c30-1d5dc0db24dd">
<table border="0" cellpadding="0" cellspacing="0" class="table table-bordered">
<colgroup>
<col width="20%"/>
<col width="80%"/>
</colgroup>
<tbody>
<tr>
<td valign="top"><a class="qTipCourse" data-url="/content/ryerson/calendar/2021-2022/courses/english/ENG/503/jcr:content/resCalendarCourseEmbed.json" href="/content/ryerson/calendar/2021-2022/courses/english/ENG/503.html">ENG 503</a></td>
<td valign="top">Science Fiction</td>
</tr>
<tr>
<td valign="top"><a class="qTipCourse" data-url="/content/ryerson/calendar/2021-2022/courses/geography/GEO/702/jcr:content/resCalendarCourseEmbed.json" href="/content/ryerson/calendar/2021-2022/courses/geography/GEO/702.html">GEO 702</a></td>
<td valign="top">Technology and the Contemporary Environme

In [20]:
sub_containers = [container for container in sub_containers if sub_containers.index(container) != 3]
len(sub_containers)

8

In [21]:
link_texts = [container.findAll("a", {"class": "qTipCourse"}) for container in sub_containers]
link_texts = [link_text.text for sub_list in link_texts for link_text in sub_list]
link_texts

['EES 612',
 'MEC 430',
 'MEC 514',
 'MEC 516',
 'MTH 510',
 'MTL 300',
 'MEC 411',
 'MEC 613',
 'MEC 616',
 'MEC 701',
 'MEC 709',
 'MEC 721',
 'MEC 722',
 'MEC 723',
 'MEC 734',
 'MEC 431',
 'MEC 740',
 'MTL 700',
 'MEC 431',
 'MEC 531',
 'MEC 617',
 'MEC 626',
 'MEC 713',
 'MEC 740',
 'MEC 751',
 'MEC 813',
 'MEC 817',
 'MTL 700',
 'CEN 800',
 'MEC 825',
 'MEC 810',
 'MEC 811',
 'MEC 816',
 'MEC 809',
 'MEC 810',
 'MEC 811',
 'MEC 816',
 'MEC 832',
 'MEC 850']

In [22]:
len(link_texts)

39

In [23]:
len(course_names)

22

## 2. Automation script to scrape all courses

In [24]:
#scrape the upper years courses

counter = 0

driver.get("https://www.ryerson.ca/calendar/2021-2022/programs/feas/mechanical/")
driver.find_element_by_class_name("resAccordionOpenAll").click()
time.sleep(3)

for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(1)
    link.click()    
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    container = page_soup.find("div", {"class": "resCalendarCourseEmbed resCalendarCourse"})
    course_code = container.find("h1").text.strip()
    course_name = container.find("h2").text.strip()
    course_desc = container.find("div", {"class": "courseDescription"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    #go to course list page
    driver.back()
    time.sleep(3)
    driver.find_element_by_class_name("resAccordionOpenAll").click()
    time.sleep(2)
        
print("Finished scraping {} courses".format(counter))

Scraped  EES 612
Scraped  MEC 430
Scraped  MEC 514
Scraped  MEC 516
Scraped  MTH 510
Scraped  MTL 300
Scraped  MEC 411
Scraped  MEC 613
Scraped  MEC 616
Scraped  MEC 701
Scraped  MEC 709
Scraped  MEC 721
Scraped  MEC 722
Scraped  MEC 723
Scraped  MEC 734
Scraped  MEC 431
Scraped  MEC 740
Scraped  MTL 700
Scraped  MEC 431
Scraped  MEC 531
Scraped  MEC 617
Scraped  MEC 626
Scraped  MEC 713
Scraped  MEC 740
Scraped  MEC 751
Scraped  MEC 813
Scraped  MEC 817
Scraped  MTL 700
Scraped  CEN 800
Scraped  MEC 825
Scraped  MEC 810
Scraped  MEC 811
Scraped  MEC 816
Scraped  MEC 809
Scraped  MEC 810
Scraped  MEC 811
Scraped  MEC 816
Scraped  MEC 832
Scraped  MEC 850
Finished scraping 39 courses


In [25]:
len(course_codes)

61

## 3. Inspect, clean and write to CSV

In [26]:
course_codes1 = []
course_names1 = []
course_descs1 = []

for i in range(len(course_codes)):
    if course_codes[i] not in course_codes1:
        course_codes1.append(course_codes[i])
        course_names1.append(course_names[i])
        course_descs1.append(course_descs[i])
    else:
        print("Duplicate: ", course_codes[i])

Duplicate:  MEC 431
Duplicate:  MEC 740
Duplicate:  MTL 700
Duplicate:  MEC 810
Duplicate:  MEC 811
Duplicate:  MEC 816


In [27]:
course_codes1

['CEN 100',
 'CEN 199',
 'CHY 102',
 'MTH 140',
 'MTH 141',
 'PCS 211',
 'CPS 188',
 'ECN 801',
 'MEC 222',
 'MTH 240',
 'MTL 200',
 'PCS 125',
 'CMN 432',
 'MEC 323',
 'MEC 325',
 'MTH 425',
 'PCS 213',
 'EES 512',
 'MEC 309',
 'MEC 311',
 'MEC 322',
 'MTH 410',
 'EES 612',
 'MEC 430',
 'MEC 514',
 'MEC 516',
 'MTH 510',
 'MTL 300',
 'MEC 411',
 'MEC 613',
 'MEC 616',
 'MEC 701',
 'MEC 709',
 'MEC 721',
 'MEC 722',
 'MEC 723',
 'MEC 734',
 'MEC 431',
 'MEC 740',
 'MTL 700',
 'MEC 531',
 'MEC 617',
 'MEC 626',
 'MEC 713',
 'MEC 751',
 'MEC 813',
 'MEC 817',
 'CEN 800',
 'MEC 825',
 'MEC 810',
 'MEC 811',
 'MEC 816',
 'MEC 809',
 'MEC 832',
 'MEC 850']

In [28]:
course_names1

['Introduction to Engineering',
 'Writing Skills',
 'General Chemistry',
 'Calculus I',
 'Linear Algebra',
 'Physics: Mechanics',
 'Computer Programming Fundamentals',
 'Principles of Engineering Economics',
 'Engineering Graphical Communication',
 'Calculus II',
 'Materials Science Fundamentals',
 'Physics: Waves and Fields',
 'Communication in the Engineering Professions',
 'Statics and Mechanics of Materials',
 'Introduction to Engineering Design',
 'Differential Equations and Vector Calculus',
 'Physics: Light and Modern Physics',
 'Electric Circuits',
 'Basic Thermodynamics',
 'Dynamics',
 'Manufacturing Fundamentals',
 'Statistics',
 'Electric Machines and Actuators',
 'Solid Mechanics I',
 'Applied Thermodynamics',
 'Fluid Mechanics I',
 'Numerical Analysis',
 'Materials Science II',
 'Mechanics of Machines',
 'Machine Design I',
 'Fluid Mechanics II',
 'Heat Transfer',
 'Control Systems',
 'Vibrations',
 'Thermal Systems Design',
 'Mechanical Systems Design',
 'Design for Manuf

In [29]:
course_descs1

['This course is aimed at familiarizing the first year students with the basic information of the academic structure and expectations. Exposure to public and worker safety and the impact of engineering activities on health as well as safety standards and safety codes will be covered. The course also stresses integration with other first year courses. The principal objectives of the course are to provide a general introduction to the field of engineering; to convey the social, professional, and ethical responsibilities of engineers and why they are important to an engineering education; to introduce the undergraduate engineering programs available at Ryerson University; and to provide a general description of the skills needed to become a practicing engineer. Case studies in engineering are used to illustrate engineering fields and scientific principles.',
 "All engineering students are required to write a mandatory Writing Skills Test (WST) administered during Orientation Week before t

In [30]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes1,
    "Course Name": course_names1,
    "Course Description": course_descs1
})

df

,Course Number,Course Name,Course Description
0,CEN 100,Introduction to Engineering,This course is aimed at familiarizing the firs...
1,CEN 199,Writing Skills,All engineering students are required to write...
2,CHY 102,General Chemistry,This course is intended for Engineering studen...
3,MTH 140,Calculus I,"Limits, continuity, differentiability, rules o..."
4,MTH 141,Linear Algebra,Systems of linear equations and matrices. Dete...
5,PCS 211,Physics: Mechanics,"Vector forces: forces along a line, coplanar f..."
6,CPS 188,Computer Programming Fundamentals,"This course introduces concepts, theory and pr..."
7,ECN 801,Principles of Engineering Economics,Engineering economics is concerned with the pr...
8,MEC 222,Engineering Graphical Communication,Introduction to technical drawing in complianc...
9,MTH 240,Calculus II,Integration techniques. L'HÃ´pital's Rule. Imp...


In [31]:
df.to_csv('Ryerson_MechEng_Core_and_Electives_(AllYears)_Courses.csv', index = False)

In [32]:
driver.quit()